
# O problema

Gerar relatórios iniciais a serem apresentados em uma reunião de investidores e acionistas, explicando a quantidade de vinhos exportados e os fatores externos que podem vir a surgir e que interferem nas análises:

1. Dados climáticos.
2. Dados demográficos.
3. Dados econômicos.
4. Dados de avaliações de vinhos.

O head de dados pediram para que você construísse uma tabela contendo as seguintes informações:

1. País de origem (Brasil).
2. País de destino.
3. Quantidade em litros de vinho exportado (utilize: 1kg = 1l).
4. Valor em US$.

Seu objetivo é dizer o montante de venda de exportação nos últimos 15 anos, separando a análise por país e trazendo quais as propecções futuras e possíveis ações para uma melhoria nas exportações. Construa gráficos atraentes e que passem a ideia central para que os acionistas e investidores possam seguir em frente com suas ações.

fontes: 

http://vitibrasil.cnpuv.embrapa.br/index.php?opcao=opt_06

https://pt.wikipedia.org/wiki/Lista_de_pa%C3%ADses_por_popula%C3%A7%C3%A3o

## Importando dados de exportação de vinhos de mesa

In [109]:
import warnings 
warnings.filterwarnings(action = 'ignore')

In [110]:
import pandas as pd
import numpy as np


dados = pd.read_csv('../dados/ExpVinho.csv', sep = ';')

In [111]:
dados.head()

,Id,País,1970,1970.1,1971,1971.1,1972,1972.1,1973,1973.1,...,2017,2017.1,2018,2018.1,2019,2019.1,2020,2020.1,2021,2021.1
0,1,Afeganistão,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,11,46
1,2,África do Sul,0,0,0,0,0,0,0,0,...,0,0,0,0,26,95,4,21,0,0
2,3,"Alemanha, República Democrática",0,0,0,0,4168,2630,12000,8250,...,15467,87702,10794,45382,3660,25467,6261,32605,2698,6741
3,4,Angola,0,0,0,0,0,0,0,0,...,10170,61680,477,709,345,1065,0,0,0,0
4,5,Anguilla,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Tratando dados

In [112]:
dados.isnull().sum().sum()

0

In [113]:
del dados['Id']
dados.head()

,País,1970,1970.1,1971,1971.1,1972,1972.1,1973,1973.1,1974,...,2017,2017.1,2018,2018.1,2019,2019.1,2020,2020.1,2021,2021.1
0,Afeganistão,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,11,46
1,África do Sul,0,0,0,0,0,0,0,0,0,...,0,0,0,0,26,95,4,21,0,0
2,"Alemanha, República Democrática",0,0,0,0,4168,2630,12000,8250,0,...,15467,87702,10794,45382,3660,25467,6261,32605,2698,6741
3,Angola,0,0,0,0,0,0,0,0,0,...,10170,61680,477,709,345,1065,0,0,0,0
4,Anguilla,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [114]:
dados.columns.get_loc('2007'), dados.columns.get_loc('2021')

(75, 103)

In [115]:
selecao = dados.columns[75:105]
selecao

Index(['2007', '2007.1', '2008', '2008.1', '2009', '2009.1', '2010', '2010.1',
       '2011', '2011.1', '2012', '2012.1', '2013', '2013.1', '2014', '2014.1',
       '2015', '2015.1', '2016', '2016.1', '2017', '2017.1', '2018', '2018.1',
       '2019', '2019.1', '2020', '2020.1', '2021', '2021.1'],
      dtype='object')

In [116]:
selecao = selecao.insert(0, 'País')
selecao

Index(['País', '2007', '2007.1', '2008', '2008.1', '2009', '2009.1', '2010',
       '2010.1', '2011', '2011.1', '2012', '2012.1', '2013', '2013.1', '2014',
       '2014.1', '2015', '2015.1', '2016', '2016.1', '2017', '2017.1', '2018',
       '2018.1', '2019', '2019.1', '2020', '2020.1', '2021', '2021.1'],
      dtype='object')

In [117]:
dados_uteis = dados[selecao]
dados_uteis.head()

,País,2007,2007.1,2008,2008.1,2009,2009.1,2010,2010.1,2011,...,2017,2017.1,2018,2018.1,2019,2019.1,2020,2020.1,2021,2021.1
0,Afeganistão,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,11,46
1,África do Sul,0,0,0,0,0,0,0,0,0,...,0,0,0,0,26,95,4,21,0,0
2,"Alemanha, República Democrática",119512,238052,265742,429970,225086,393482,27715,138666,36070,...,15467,87702,10794,45382,3660,25467,6261,32605,2698,6741
3,Angola,25931,49753,25721,71083,54786,84235,33557,189891,13889,...,10170,61680,477,709,345,1065,0,0,0,0
4,Anguilla,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [118]:
from unidecode import unidecode


#função para remover acentos
def remover_acentos(texto):
    texto_sem_acentos = unidecode(texto)
    return texto_sem_acentos

#função para padronizar os nomes dos países

def padronizar_nome_pais(nome_pais):
    padroes = {
        'Alemanha, Republica Democratica': 'Alemanha',
        'Alemanha, Republica Democratica da': 'Alemanha',
        'Belice': 'Belize',
        'Belize': 'Belize',
        'Camaroes': 'Camaroes',
        'Camores': 'Camaroes',
        'Coreia do Sul, Republica da': 'Coreia do Sul',
        'Coreia do Sul, Republica': 'Coreia do Sul',
        'Coreia do Sul': 'Coreia do Sul',
        'Coreia, Republica Sul': 'Coreia do Sul',
        'Coreia, Republica Sul': 'Coreia do Sul',
        'Dominica': 'Dominica',
        'Dominica, Ilha de': 'Dominica',
        'Eslovaca, Republica': 'Eslovaquia',
        'Falkland (Ilhas Malvinas)': 'Ilhas Malvinas',
        'Falkland (Malvinas)': 'Ilhas Malvinas',
        'Filanldia': 'Finlandia',
        'Finlandia': 'Finlandia',
        'Guine Bissau': 'Guine-Bissau',
        'Guine Bissau': 'Guine-Bissau',
        'Cayman, Ilhas': 'Ilhas Cayman',
        'Marshall, Ilhas': 'Ilhas Marshall',
        'Paises Baixos': 'Holanda',
        'Paises Baixos (Holanda)': 'Holanda',
        'Russia': 'Russia',
        'Russia, Federacao da': 'Russia',
        'Republica Federativa da Russia': 'Russia',
        'Taiwan (FORMOSA)': 'Taiwan',
        'Taiwan (Formosa)': 'Taiwan',
        'Trindade e Tobago': 'Trindade e Tobago',
        'Trinidade Tobago': 'Trindade e Tobago',
        'Trinidade e Tobago': 'Trindade e Tobago',
        'Tcheca, Republica': 'Republica Tcheca',
        'Georgia do Sul e Sandwich do Sul, Ilhas': 'Ilhas Georgia do Sul e Sandwich do Sul'
    }

    if nome_pais in padroes:
        return padroes[nome_pais]
    else:
        return nome_pais
        
dados_uteis['País'] = dados_uteis['País'].apply(remover_acentos)
dados_uteis['País'] = dados_uteis['País'].apply(padronizar_nome_pais)

In [119]:
dados_uteis['País']

0        Afeganistao
1      Africa do Sul
2           Alemanha
3             Angola
4           Anguilla
           ...      
119           Tuvalu
120          Uruguai
121          Vanuatu
122        Venezuela
123           Vietna
Name: País, Length: 124, dtype: object

In [120]:
quantidade = [i for i in dados_uteis.columns if len(i) <= 4 and i != 'País']
quantidade

['2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021']

In [121]:
valor = [i for i in dados_uteis.columns if len(i) > 4 and i != 'País']
valor

['2007.1',
 '2008.1',
 '2009.1',
 '2010.1',
 '2011.1',
 '2012.1',
 '2013.1',
 '2014.1',
 '2015.1',
 '2016.1',
 '2017.1',
 '2018.1',
 '2019.1',
 '2020.1',
 '2021.1']

In [122]:
dados_uteis['quantidade_total'] = dados_uteis[quantidade].sum(axis = 1)
dados_uteis['valor_total'] = dados_uteis[valor].sum(axis = 1)
dados_uteis.head()

,País,2007,2007.1,2008,2008.1,2009,2009.1,2010,2010.1,2011,...,2018,2018.1,2019,2019.1,2020,2020.1,2021,2021.1,quantidade_total,valor_total
0,Afeganistao,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,11,46,11,46
1,Africa do Sul,0,0,0,0,0,0,0,0,0,...,0,0,26,95,4,21,0,0,30,116
2,Alemanha,119512,238052,265742,429970,225086,393482,27715,138666,36070,...,10794,45382,3660,25467,6261,32605,2698,6741,1020933,2739079
3,Angola,25931,49753,25721,71083,54786,84235,33557,189891,13889,...,477,709,345,1065,0,0,0,0,190731,621181
4,Anguilla,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [123]:
quantidade.insert(0, 'País')
dados_quantidade = dados_uteis[quantidade]
dados_quantidade.rename(columns = {'País': 'pais'}, inplace = True)
dados_quantidade.head()

,pais,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Afeganistao,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11
1,Africa do Sul,0,0,0,0,0,0,0,0,0,0,0,0,26,4,0
2,Alemanha,119512,265742,225086,27715,36070,8189,61699,213348,10680,14012,15467,10794,3660,6261,2698
3,Angola,25931,25721,54786,33557,13889,2833,1573,12182,1908,7359,10170,477,345,0,0
4,Anguilla,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [124]:
valor.insert(0, 'País')
dados_valor = dados_uteis[valor]
dados_valor.columns = quantidade
dados_valor.rename(columns = {'País': 'pais'}, inplace = True)
dados_valor.head()

,pais,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Afeganistao,0,0,0,0,0,0,0,0,0,0,0,0,0,0,46
1,Africa do Sul,0,0,0,0,0,0,0,0,0,0,0,0,95,21,0
2,Alemanha,238052,429970,393482,138666,144150,56342,265978,761653,44780,68109,87702,45382,25467,32605,6741
3,Angola,49753,71083,84235,189891,69001,8861,9300,23124,17089,35390,61680,709,1065,0,0
4,Anguilla,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [125]:
dados_quantidade_fechado = dados_quantidade.set_index('pais')
dados_quantidade_fechado['quantidade_total'] = dados_quantidade_fechado.sum(axis = 1)
dados_quantidade_fechado.head()

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,quantidade_total
pais,,,,,,,,,,,,,,,,
Afeganistao,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,11
Africa do Sul,0,0,0,0,0,0,0,0,0,0,0,0,26,4,0,30
Alemanha,119512,265742,225086,27715,36070,8189,61699,213348,10680,14012,15467,10794,3660,6261,2698,1020933
Angola,25931,25721,54786,33557,13889,2833,1573,12182,1908,7359,10170,477,345,0,0,190731
Anguilla,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [126]:
dados_valor_fechado = dados_valor.set_index('pais')
dados_valor_fechado['valor_total'] = dados_valor_fechado.sum(axis = 1)
dados_valor_fechado.head()

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,valor_total
pais,,,,,,,,,,,,,,,,
Afeganistao,0,0,0,0,0,0,0,0,0,0,0,0,0,0,46,46
Africa do Sul,0,0,0,0,0,0,0,0,0,0,0,0,95,21,0,116
Alemanha,238052,429970,393482,138666,144150,56342,265978,761653,44780,68109,87702,45382,25467,32605,6741,2739079
Angola,49753,71083,84235,189891,69001,8861,9300,23124,17089,35390,61680,709,1065,0,0,621181
Anguilla,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Tratando dados para o melt

In [127]:
dados_valor_fechado_temp = dados_valor_fechado.drop('valor_total', axis = 1)
dados_valor_fechado_temp.head()

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
pais,,,,,,,,,,,,,,,
Afeganistao,0,0,0,0,0,0,0,0,0,0,0,0,0,0,46
Africa do Sul,0,0,0,0,0,0,0,0,0,0,0,0,95,21,0
Alemanha,238052,429970,393482,138666,144150,56342,265978,761653,44780,68109,87702,45382,25467,32605,6741
Angola,49753,71083,84235,189891,69001,8861,9300,23124,17089,35390,61680,709,1065,0,0
Anguilla,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [128]:
dados_valor_aberto = dados_valor_fechado_temp.T
dados_valor_aberto.head()

pais,Afeganistao,Africa do Sul,Alemanha,Angola,Anguilla,Antigua e Barbuda,Antilhas Holandesas,Argentina,Aruba,Australia,...,Republica Tcheca,Togo,Trindade e Tobago,Tunisia,Turquia,Tuvalu,Uruguai,Vanuatu,Venezuela,Vietna
2007,0,0,238052,49753,0,0,17949,0,0,281,...,189620,11,0,0,0,0,0,0,27370,0
2008,0,0,429970,71083,0,0,22908,0,0,99280,...,64709,7800,0,0,0,0,0,0,0,0
2009,0,0,393482,84235,0,0,10651,4523,0,9195,...,43902,0,0,0,0,0,0,0,0,2143
2010,0,0,138666,189891,0,0,12808,0,0,17960,...,5285,0,0,0,0,0,2929,0,0,9977
2011,0,0,144150,69001,0,0,10188,55460,1680,40704,...,37271,0,0,0,0,0,4404,0,0,0


In [129]:
dados_valor_aberto.reset_index(inplace = True)
dados_valor_aberto.head()

pais,index,Afeganistao,Africa do Sul,Alemanha,Angola,Anguilla,Antigua e Barbuda,Antilhas Holandesas,Argentina,Aruba,...,Republica Tcheca,Togo,Trindade e Tobago,Tunisia,Turquia,Tuvalu,Uruguai,Vanuatu,Venezuela,Vietna
0,2007,0,0,238052,49753,0,0,17949,0,0,...,189620,11,0,0,0,0,0,0,27370,0
1,2008,0,0,429970,71083,0,0,22908,0,0,...,64709,7800,0,0,0,0,0,0,0,0
2,2009,0,0,393482,84235,0,0,10651,4523,0,...,43902,0,0,0,0,0,0,0,0,2143
3,2010,0,0,138666,189891,0,0,12808,0,0,...,5285,0,0,0,0,0,2929,0,0,9977
4,2011,0,0,144150,69001,0,0,10188,55460,1680,...,37271,0,0,0,0,0,4404,0,0,0


In [130]:
dados_valor_aberto = dados_valor_aberto.melt(id_vars = ['index'], value_vars = dados_valor_aberto.columns)
dados_valor_aberto.columns = ['ano', 'pais', 'valor']

In [131]:
dados_valor_aberto.head()

,ano,pais,valor
0,2007,Afeganistao,0
1,2008,Afeganistao,0
2,2009,Afeganistao,0
3,2010,Afeganistao,0
4,2011,Afeganistao,0


In [132]:
dados_quantidade_fechado_temp = dados_quantidade_fechado.drop('quantidade_total', axis = 1)
dados_quantidade_fechado_temp.head()

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
pais,,,,,,,,,,,,,,,
Afeganistao,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11
Africa do Sul,0,0,0,0,0,0,0,0,0,0,0,0,26,4,0
Alemanha,119512,265742,225086,27715,36070,8189,61699,213348,10680,14012,15467,10794,3660,6261,2698
Angola,25931,25721,54786,33557,13889,2833,1573,12182,1908,7359,10170,477,345,0,0
Anguilla,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [133]:
dados_quantidade_aberto = dados_quantidade_fechado_temp.T
dados_quantidade_aberto.head()

pais,Afeganistao,Africa do Sul,Alemanha,Angola,Anguilla,Antigua e Barbuda,Antilhas Holandesas,Argentina,Aruba,Australia,...,Republica Tcheca,Togo,Trindade e Tobago,Tunisia,Turquia,Tuvalu,Uruguai,Vanuatu,Venezuela,Vietna
2007,0,0,119512,25931,0,0,17025,0,0,60,...,85303,11,0,0,0,0,0,0,20700,0
2008,0,0,265742,25721,0,0,17938,0,0,218726,...,17135,5160,0,0,0,0,0,0,0,0
2009,0,0,225086,54786,0,0,8235,162,0,1014,...,9269,0,0,0,0,0,0,0,0,743
2010,0,0,27715,33557,0,0,9810,0,0,1823,...,1091,0,0,0,0,0,914,0,0,8820
2011,0,0,36070,13889,0,0,7335,13253,900,3632,...,6846,0,0,0,0,0,1238,0,0,0


In [134]:
dados_quantidade_aberto.reset_index(inplace = True)
dados_quantidade_aberto.head()

pais,index,Afeganistao,Africa do Sul,Alemanha,Angola,Anguilla,Antigua e Barbuda,Antilhas Holandesas,Argentina,Aruba,...,Republica Tcheca,Togo,Trindade e Tobago,Tunisia,Turquia,Tuvalu,Uruguai,Vanuatu,Venezuela,Vietna
0,2007,0,0,119512,25931,0,0,17025,0,0,...,85303,11,0,0,0,0,0,0,20700,0
1,2008,0,0,265742,25721,0,0,17938,0,0,...,17135,5160,0,0,0,0,0,0,0,0
2,2009,0,0,225086,54786,0,0,8235,162,0,...,9269,0,0,0,0,0,0,0,0,743
3,2010,0,0,27715,33557,0,0,9810,0,0,...,1091,0,0,0,0,0,914,0,0,8820
4,2011,0,0,36070,13889,0,0,7335,13253,900,...,6846,0,0,0,0,0,1238,0,0,0


In [135]:
dados_quantidade_aberto = dados_quantidade_aberto.melt(id_vars = ['index'], value_vars = dados_quantidade_aberto.columns)
dados_quantidade_aberto.columns = ['ano', 'pais', 'quantidade']

In [136]:
dados_quantidade_aberto.head()

,ano,pais,quantidade
0,2007,Afeganistao,0
1,2008,Afeganistao,0
2,2009,Afeganistao,0
3,2010,Afeganistao,0
4,2011,Afeganistao,0


In [137]:
dados_uteis_aberto = dados_quantidade_aberto.join(dados_valor_aberto['valor'])
dados_uteis_aberto.head()

,ano,pais,quantidade,valor
0,2007,Afeganistao,0,0
1,2008,Afeganistao,0,0
2,2009,Afeganistao,0,0
3,2010,Afeganistao,0,0
4,2011,Afeganistao,0,0


In [138]:
dados_uteis_aberto.columns = ['ano', 'pais', 'quantidade_exportacao', 'valor_exportacao']
dados_uteis_aberto.head()

,ano,pais,quantidade_exportacao,valor_exportacao
0,2007,Afeganistao,0,0
1,2008,Afeganistao,0,0
2,2009,Afeganistao,0,0
3,2010,Afeganistao,0,0
4,2011,Afeganistao,0,0


In [139]:
dados_uteis_aberto.query('quantidade_exportacao == 0 and valor_exportacao > 0') 

,ano,pais,quantidade_exportacao,valor_exportacao
358,2020,Camaroes,0,178


In [140]:
dados_uteis_aberto['valor_exportacao_por_litro'] = (dados_uteis_aberto['valor_exportacao'] / dados_uteis_aberto['quantidade_exportacao']).round(2)

In [141]:
dados_uteis_aberto.head()

,ano,pais,quantidade_exportacao,valor_exportacao,valor_exportacao_por_litro
0,2007,Afeganistao,0,0,NaN
1,2008,Afeganistao,0,0,NaN
2,2009,Afeganistao,0,0,NaN
3,2010,Afeganistao,0,0,NaN
4,2011,Afeganistao,0,0,NaN


In [142]:
dados_uteis_aberto['valor_exportacao_por_litro'] = dados_uteis_aberto['valor_exportacao_por_litro'].fillna(0)
dados_uteis_aberto

,ano,pais,quantidade_exportacao,valor_exportacao,valor_exportacao_por_litro
0,2007,Afeganistao,0,0,0.00
1,2008,Afeganistao,0,0,0.00
2,2009,Afeganistao,0,0,0.00
3,2010,Afeganistao,0,0,0.00
4,2011,Afeganistao,0,0,0.00
...,...,...,...,...,...
1855,2017,Vietna,0,0,0.00
1856,2018,Vietna,0,0,0.00
1857,2019,Vietna,20,32,1.60
1858,2020,Vietna,86,584,6.79


In [143]:
dados_uteis_aberto.query('valor_exportacao_por_litro == inf') #verificando se constam valores resultados de divisão com dividendo 0

,ano,pais,quantidade_exportacao,valor_exportacao,valor_exportacao_por_litro
358,2020,Camaroes,0,178,inf


In [144]:
dados_uteis_aberto.replace(np.inf, 0, inplace = True)
dados_uteis_aberto.query('valor_exportacao_por_litro == inf')

,ano,pais,quantidade_exportacao,valor_exportacao,valor_exportacao_por_litro


In [145]:
dados_uteis_aberto.isnull().sum()

ano                           0
pais                          0
quantidade_exportacao         0
valor_exportacao              0
valor_exportacao_por_litro    0
dtype: int64

## Exportando dados

In [146]:
dados_uteis_aberto.to_csv('../dados/exportacao_util.csv', index = False)